### Use RefSeq viruses to perform taxonomic clustering. 
- Download Refseq viruses that infect Archeae and Bacteria
- Translate proteins using a specific translator, prodigal-gv
- One proteome for each viral contig (/home/ctbrown/scratch3/2024-annie-virus-protein-sketch)
- Concatenate proteome with ICTV, then cluster. And whatever cluster with something from ICTV, gets annotated.
AND/OR
- fastmultigather for each item in pairwise against ictv. Then run tax annotate. Then cluster alone, w/o ICTV. For each cluster, what does sourmash tax annotate say? And they all corresponding within the cluster. If yes –> annotate, if all correspond at higher tax level, annotate at that.
- Run Genomad too, to compare outputs

Questions
- Family level db?
- Genus level?
- scale: 2,5,10,100. protein. k=10,7,12.
- Or iteratively, go dna then protein. everything that doesnt get scored gets thrown to protein

#### Download NCBI genomes (n=4568 for bacteria, 102 archaea)
- go to https://www.ncbi.nlm.nih.gov/genomes/GenomesGroup.cgi?taxid=10239
- select host bacteria and host archaea.
- Download nucleotide records
- get taxonomy for all

In [ ]:
# Clean the headers because spaces and we only need the accession number
mamba activate bbmap
reformat.sh in=240627_RefSeq_phagegenomes.fa out=240627_RefSeq_phagegenomes.rn.fa trd

In [ ]:
# Use the INSDSeq xml files for taxonomy. 
see get_tax notebook

In [ ]:
# split the multifasta into individual sequences
mkdir contigs
cd ./contigs
awk '/^>/ {OUT=substr($0,2) ".fa"}; OUT {print >OUT}' ../240627_RefSeq_phagegenomes.rn.fa 

In [ ]:
# Use prodigal-gv for annotation
srun --account=ctbrowngrp -p bmm -J prodigal-gv -t 24:00:00 -c 1 --mem=50gb --pty bash

mamba activate prodigal-gv
prodigal-gv 
prodigal-gv -p meta -i 240627_RefSeq_phagegenomes.rn.fasta \
-a ../../results/240627_RefSeq.proteins.faa 


In [ ]:
# Run the custom_sketch.py for protein sketches of each genome.
# remove spaces from predicted proteins
cut -d ' ' -f1 240627_RefSeq.proteins.faa > 240627_RefSeq.proteins.ns.faa 

# run custom_sketch. Scale of 2, k=7,10,12
python ../workflow/scripts/custom-sketch.py \
240627_RefSeq.proteins.ns.faa --ksize 12 --scaled 2 -o 240627_RefSeq.proteins.k10.sc2.zip

# can I concatenate those?
sourmash sig cat *sc2.zip -o 240627_RefSeq.proteins.zip

#### Sourmash clustering
Multiple ways possible:
1. Fastmultigather against ICTV --> tax annotate the results. Pairwise within own viral sequences, cluster the output. Compare each clusters tax annotations from fmg
2. pairwise comparison against ICTV. After this, do clustering with the output csv. Whatever get clustered with an ICTV virus gets annotated


Need to do this at various k and scale sizes
- k: 7,10,12
- scale: 2,10,100



In [ ]:
# run snakemake for fastmultigather
srun --account=ctbrowngrp -p high2 -J fmg -t 1:00:00 -c 50 --mem=60gb --pty bash
mamba activate branchwater

snakemake --use-conda --resources mem_mb=60000 --rerun-triggers mtime \
-c 50 --rerun-incomplete -k -n

In [ ]:
# after fmg, use tax annotate
# use --from-file with a list of paths to fmg output. 


# tax db /home/ntpierce/2023-spillover/output.vmr/vmr_MSL38_v1.taxonomy.csv
sourmash tax metagenome -g {input.csv} -t {TAXDB} > {output.taxout}
sourmash tax annotate \
-g ffea67de977c0a3f8a380cf0806af2f6.sig.gz.gather.csv -t 

In [ ]:
# then pairwise cluster (within own sequences), for each different ksize, scalevalue
mamba activate branchwater
sourmash scripts pairwise \
{sig_paths} -k 12 -s 2 -m protein -c {threads} \
-a --write-all -o {output}

In [ ]:
# now with all output, try to get a consensus taxonomy. 

In [ ]:
# 2nd method to try:
# Concatenate proteome with ICTV, then cluster. And whatever clusters with something from ICTV, gets annotated.
# concat
sourmash sig cat 240627_RefSeq.proteins.zip ../../resources/vmr_MSL38_v1.protein.zip -o RefSeq_vmr.zip

# cluster takes the output of pairwise or multisearch (same thing)
sourmash scripts pairwise \
240627_RefSeq.proteins.zip \
../resources/vmr_MSL38_v1.protein.zip -k 12 -s 100 \
-c 12 -o multisearch.RS.ICTV.k12.s100.csv -m protein


sourmash scripts pairwise --write-all -a -c 12

# cluster
sourmash cluster

# pairwise takes in multiple files, as long as they aren't zips. 
sourmash scripts multisearch f2.zip f1.zip -k 12 -s 100 -o test.csv -m protein -a

In [ ]:
python ../workflow/scripts/annotate-clusters.py \
./cluster/refseq.k7.s2.ani0.95.csv \
-t tax/csv_concat/refseq.k7s2.csv \
-o ./tax_clust/ --ictv 

In [ ]:
python ../workflow/scripts/annotate-clusters.py \
./cluster/refseq.k7.s2.ani0.95.csv \
-t tax/refseq.k7.s2/fffa284ebe6dc9f64c4733d9fea6bc92.sig.gz.gather.with-lineages.csv \
-o ./tax_clust/ --ictv